In [7]:
import os
import gzip
import numpy as np
import json

In [8]:
Ns = 10                 # number of stocks
Nt = 9                  # number of time steps                # Number of bits in the binary representation of the stock quantities
Nq = 1                  # Number of bits in the binary representation of the stock quantities
Ntot = Ns * Nt
d = 2**Nq               # Physical index dimension
K = 15                  # Budget constraint

def tn_to_index(t, n):
    i = 0
    for t_prime in range(Nt):
        for n_prime in range(Ns):
            if (n_prime, t_prime) == (n, t):
                return i
            i += 1
    return -1  # Should never reach here if inputs are valid

def index_to_tn(i):
    n = i % Ns
    t = (i // Ns) % Nt
    return t, n


## Load data

In [9]:
path = f"./data/instances/po_a0{Ns}_t{Nt + 1}_orig"

path_covariance = os.path.join(path, "covariance_matrices.txt.gz")
path_prices = os.path.join(path, "stock_prices.txt.gz")

print(path_covariance)
print(path_prices)
skiplines = 9

# read
with gzip.open(path_covariance, 'rt') as f:
    covariance_matrices = []
    for line in f:
        row = [x for x in line.strip().split()]
        covariance_matrices.append(row)

with gzip.open(path_prices, 'rt') as f:
    stock_prices = []
    for line in f:
        stock_prices.append([x for x in line.strip().split()])

covariance_matrices = covariance_matrices[skiplines:]
stock_prices = stock_prices[skiplines:]

./data/instances/po_a010_t10_orig/covariance_matrices.txt.gz
./data/instances/po_a010_t10_orig/stock_prices.txt.gz


## Map stock ticker to numerical index

In [10]:
# Create mapping from stock id to index
stock_to_index = {}
index_to_stock = {}

for i, price in enumerate(stock_prices):
    if int(price[0]) == 0:
        stock_to_index[price[1]] = i
        index_to_stock[i] = price[1]

stock_to_index

{'AAPL': 0,
 'NVDA': 1,
 'MSFT': 2,
 'GOOG': 3,
 'GOOGL': 4,
 'AMZN': 5,
 'META': 6,
 'TSLA': 7,
 'AVGO': 8,
 'WMT': 9}

## Create tensors $\mu_{t,n}$ and $\Sigma_{t, n', n}$

In [11]:
prices = np.zeros((Nt + 1, Ns))
sigma = np.zeros((Nt + 1, Ns, Ns))

for row in stock_prices:
    t = int(row[0])
    stock_id = row[1]
    i = stock_to_index[stock_id]
    prices[t, i] = float(row[2])

for row in covariance_matrices:
    t = int(row[0])
    stock_id1 = row[1]
    stock_id2 = row[2]
    i = stock_to_index[stock_id1]
    j = stock_to_index[stock_id2]
    sigma[t, i, j] = float(row[3])
    sigma[t, j, i] = float(row[3])  # symmetric

# Drop first day of covariance matrices (no returns on day 0)
sigma = sigma[1:]

# Calculate log returns
mu = (prices[1:] - prices[:-1]) / prices[:-1]  # bare returns
mu = np.log(1 + mu)  # log returns

## MIP

In [12]:
gamma = .5  # risk aversion parameter
zeta = .1  # transaction cost parameter
rho = .1  # penalty parameter for budget constraint

J = {}  # interaction terms
h = {}  # local field terms
h2 = {}  # local field terms (squared)

# (A) Return term
for i in range(Ntot):
    t_i, n_i = index_to_tn(i)
    h[i] = h.get(i, 0) - mu[t_i, n_i] / K

# (B) Risk term
for t in range(Nt):
    indices_t = [tn_to_index(t, n) for n in range(Ns)]
    for i in range(len(indices_t)):
        for j in range(len(indices_t)):
            idx_i = indices_t[i]
            idx_j = indices_t[j]
            t_i, n_i = index_to_tn(idx_i)
            t_j, n_j = index_to_tn(idx_j)
            if n_i == n_j:
                h2[idx_i] = h2.get(idx_i, 0) + 0.5 * gamma * sigma[t_i, n_i, n_j] / K**2
            elif n_i != n_j:
                J[(idx_i, idx_j)] = J.get((idx_i, idx_j), 0) + 0.5 * gamma * sigma[t_i, n_i, n_j] / K**2

# (C) Transaction cost term
for t in range(Nt-1):
    indices_t = [tn_to_index(t, n) for n in range(Ns)]
    indices_tp1 = [tn_to_index(t + 1, n) for n in range(Ns)]

    for i in range(len(indices_t)):
        idx_i = indices_t[i]
        h2[idx_i] = h2.get(idx_i, 0) + zeta / K**2

    for i in range(len(indices_tp1)):
        idx_i = indices_tp1[i]
        h2[idx_i] = h2.get(idx_i, 0) + zeta / K**2

    for i in range(len(indices_t)):
        for j in range(len(indices_tp1)):
            idx_i = indices_t[i]
            idx_j = indices_tp1[j]
            t_i, n_i = index_to_tn(idx_i)
            t_j, n_j = index_to_tn(idx_j)
            if n_i == n_j:
                J[(idx_i, idx_j)] = J.get((idx_i, idx_j), 0) - 2.0 * zeta / K**2
        
# (D) Budget constraint term
for t in range(Nt):
    indices_t = [tn_to_index(t, n) for n in range(Ns)]

    for i in range(len(indices_t)):
        for j in range(len(indices_t)):
            idx_i = indices_t[i]
            idx_j = indices_t[j]
            t_i, n_i = index_to_tn(idx_i)
            t_j, n_j = index_to_tn(idx_j)
            
            if n_i == n_j:
                h2[idx_i] = h2.get(idx_i, 0) + rho / K**2
            elif n_i != n_j:
                J[(idx_i, idx_j)] = J.get((idx_i, idx_j), 0) + rho / K**2

        h[idx_i] = h.get(idx_i, 0) - 2.0 * rho / K

# Convert J to upper triangular form by making (i,j) = (i,j) + (j,i) and removing (j,i) if j > i
J_upper = {}
for (i, j), value in J.items():
    if i < j:
        J_upper[(i, j)] = J_upper.get((i, j), 0) + value
    elif i > j:
        J_upper[(j, i)] = J_upper.get((j, i), 0) + value
    else:
        # Diagonal terms should not exist in J for Ising model
        pass
J = J_upper

# Save as JSON
with open(f"./data/MIP/MIP_Ns{Ns}_Nt{Nt}_Nq{Nq}_K{K}_gamma{gamma}_zeta{zeta}_rho{rho}.json", "w") as f:
    json.dump({"J": {f"{k[0]},{k[1]}": v for k, v in J.items()}, "h": {str(k): v for k, v in h.items()}, "h2": {str(k): v for k, v in h2.items()}}, f, indent=4)

J

{(0, 1): np.float64(0.0008888577956235065),
 (0, 2): np.float64(0.0008890047754438744),
 (0, 3): np.float64(0.0008891176862683089),
 (0, 4): np.float64(0.0008891195483842721),
 (0, 5): np.float64(0.0008890005264631269),
 (0, 6): np.float64(0.000888980366516692),
 (0, 7): np.float64(0.0008892301894732587),
 (0, 8): np.float64(0.0008888236765499137),
 (0, 9): np.float64(0.0008888580020000452),
 (1, 2): np.float64(0.0008893806486861858),
 (1, 3): np.float64(0.0008888442162332351),
 (1, 4): np.float64(0.0008888753962792998),
 (1, 5): np.float64(0.0008895865254875925),
 (1, 6): np.float64(0.0008899887228915229),
 (1, 7): np.float64(0.0008891046961086689),
 (1, 8): np.float64(0.0008905366101352052),
 (1, 9): np.float64(0.0008889765227209189),
 (2, 3): np.float64(0.0008890768362334556),
 (2, 4): np.float64(0.0008890826121039424),
 (2, 5): np.float64(0.0008891709156599416),
 (2, 6): np.float64(0.0008892199413065539),
 (2, 7): np.float64(0.0008889447624333874),
 (2, 8): np.float64(0.00088935360